In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json

In [29]:
url = 'https://udn.com/news/index'

response = requests.get(url)
print(response.status_code)
if response.status_code == 200:
  soup = BeautifulSoup(response.text, 'html.parser')

200


In [30]:
#擷取即時和熱門新聞
b_tags = soup.find_all('span', class_="tab-link__title")
for tag in b_tags:
    if(tag.string!=None):
        print(tag.string)


北市松山民宅暗夜竄火 警消撲救疏散5人 起火原因待查
體操／世界盃杜哈站 李智凱鞍馬預賽第1續搶奧運資格
謝國樑食安三箭 營養午餐每人增20元、待遇追加1942萬
日本國民美食漢堡排為何突然爆紅？日人：都是從它開始
龍華部隊工兵過太爽？上網豪賭39天輸到脫褲  下場曝
萬里浮屍頭套垃圾袋雙腳綁 家屬：不知為何這樣盼查清
洪秀柱：7月將率200名台灣青年赴浙江參訪
北榮名醫涉收回扣 院長陳威明發信給全院同仁說這件事
中美經濟工作組會議 雙方再次就工業產能過剩的議題交鋒
藍營大咖爆家暴慣犯 國民黨回應了
7月底再有四所大專校院退場 教育部：5月分發安置近千學生
警追毒蟲跑到快斷氣 外送員催油門一掌巴下去抓到人
影／桃園3度跳電 張善政轟蔡英文能源政策「荒腔走板」
大甲媽回鑾鎮瀾宮副董出拳揍人！顏清標出面當公親 挨打接轎男撤告 
00929配息0.2元除息前更大跌讓大家上車！ 存股達人：真是太棒了
爆徐巧芯老公特權免兵役 劉彥澧伸右小腿拿證據要溫朗東道歉
動盪中仍「錢途亮眼」！近期財運爆棚的3大生肖
台鐵大便男落網！稱當下真的太想「方便」 扔屎抹屎畫面曝


In [31]:
#dataframe 和 dict 
list1=[]
list2=[]
list3=[]

b_tags = soup.find_all('a', class_="tab-link")
for tag in b_tags:
    c_tag = tag.find('span', class_="tab-link__title")
    if(c_tag.string!=None):
        #print(c_tag.string)
        list1.append(c_tag.string)
        if 'href' in tag.attrs:
            
            #切換至連結的頁面
            new_url = tag.get('href')
            response = requests.get(new_url)
            #print(new_url)
            
            if response.status_code == 200:
                soupp = BeautifulSoup(response.text, 'html.parser')
                
                #找關鍵字
                d_tags = soupp.find_all('a', class_="btn btn-border btn-border--blue btn-keyword tag")
                str=""
                for tagg in d_tags:
                    #print(tagg.string)
                    if(str!=""):
                        str=str+"、"+tagg.string
                    else:
                        str=tagg.string
                list2.append(str)
                
                #地點(正則化)
                e_tags = soupp.find_all('span', class_="article-content__author")
                pattern = r'／([^／]+)即時報導'
                for taggg in e_tags:
                    list3.append( re.findall(pattern, taggg.text))
                    #print(list3[-1])
                
        

In [32]:
#dataframe
df = pd.DataFrame({
    'Title': list1,
    'key word': list2,
    'location':list3
})
df

,Title,key word,location
0,北市松山民宅暗夜竄火 警消撲救疏散5人 起火原因待查,北市、建築,[台北]
1,體操／世界盃杜哈站 李智凱鞍馬預賽第1續搶奧運資格,李智凱、林育信,[]
2,謝國樑食安三箭 營養午餐每人增20元、待遇追加1942萬,營養午餐、營養師、食安,[基隆]
3,日本國民美食漢堡排為何突然爆紅？日人：都是從它開始,日本、漢堡、美食,[]
4,龍華部隊工兵過太爽？上網豪賭39天輸到脫褲 下場曝,憲兵、桃園、賭博,[桃園]
5,萬里浮屍頭套垃圾袋雙腳綁 家屬：不知為何這樣盼查清,塑膠袋、萬里,[新北]
6,洪秀柱：7月將率200名台灣青年赴浙江參訪,台海、洪秀柱、青年,[]
7,北榮名醫涉收回扣 院長陳威明發信給全院同仁說這件事,回扣、醫學、醫師,[台北]
8,中美經濟工作組會議 雙方再次就工業產能過剩的議題交鋒,溝通、美國,[]
9,藍營大咖爆家暴慣犯 國民黨回應了,國民黨、家暴、未成年,[台北]


In [33]:
#dict
dict = df.to_dict('dict')
dict

{'Title': {0: '北市松山民宅暗夜竄火 警消撲救疏散5人 起火原因待查',
  1: '體操／世界盃杜哈站 李智凱鞍馬預賽第1續搶奧運資格',
  2: '謝國樑食安三箭 營養午餐每人增20元、待遇追加1942萬',
  3: '日本國民美食漢堡排為何突然爆紅？日人：都是從它開始',
  4: '龍華部隊工兵過太爽？上網豪賭39天輸到脫褲  下場曝',
  5: '萬里浮屍頭套垃圾袋雙腳綁 家屬：不知為何這樣盼查清',
  6: '洪秀柱：7月將率200名台灣青年赴浙江參訪',
  7: '北榮名醫涉收回扣 院長陳威明發信給全院同仁說這件事',
  8: '中美經濟工作組會議 雙方再次就工業產能過剩的議題交鋒',
  9: '藍營大咖爆家暴慣犯 國民黨回應了',
  10: '7月底再有四所大專校院退場 教育部：5月分發安置近千學生',
  11: '警追毒蟲跑到快斷氣 外送員催油門一掌巴下去抓到人',
  12: '影／桃園3度跳電 張善政轟蔡英文能源政策「荒腔走板」',
  13: '大甲媽回鑾鎮瀾宮副董出拳揍人！顏清標出面當公親 挨打接轎男撤告 ',
  14: '00929配息0.2元除息前更大跌讓大家上車！ 存股達人：真是太棒了',
  15: '爆徐巧芯老公特權免兵役 劉彥澧伸右小腿拿證據要溫朗東道歉',
  16: '動盪中仍「錢途亮眼」！近期財運爆棚的3大生肖',
  17: '台鐵大便男落網！稱當下真的太想「方便」 扔屎抹屎畫面曝'},
 'key word': {0: '北市、建築',
  1: '李智凱、林育信',
  2: '營養午餐、營養師、食安',
  3: '日本、漢堡、美食',
  4: '憲兵、桃園、賭博',
  5: '塑膠袋、萬里',
  6: '台海、洪秀柱、青年',
  7: '回扣、醫學、醫師',
  8: '溝通、美國',
  9: '國民黨、家暴、未成年',
  10: '大同技術學院、大學退場、少子化、教育部、明道大學、環球科大、立法院、高教',
  11: '外送員、影片、毒品',
  12: '台電、賴清德',
  13: '大甲媽祖、遶境、鎮瀾宮、顏清標',
  14: '00878國泰永續高股息、00929復華台灣科技優息ETF、均價、大盤、除息',
  15: '免役、兵役、溫朗東',
  16

In [34]:
df.to_csv('News.csv', index=False, encoding='utf-8-sig')

In [35]:
# 將字典寫入JSON文件
with open('News.json', 'w', encoding='utf-8') as f:    
  json.dump(dict, f, ensure_ascii=False, indent=4)